In [ ]:
# Procesamiento Básico
# proceso para reducir las bases 
# FAC


In [10]:
import pandas as pd
import os

def analizar_poblacion_por_edad_y_genero(year, quarter):
    """
    Carga los microdatos sociodemográficos de la ENOE para un periodo específico
    y calcula la población estimada agrupada por edad y género.

    Args:
        year (int): El año a analizar.
        quarter (int): El trimestre a analizar (1-4).
        
    Returns:
        pd.DataFrame: Un DataFrame con la población estimada por edad y género,
                      o None si el archivo no se encuentra.
    """
    print(f"--- Analizando datos para {year} T{quarter} ---")
    
    # --- 1. Construir la ruta del archivo ---
    # Nota: Los nombres de archivo dentro del zip pueden variar y a menudo están en minúsculas.
    # El patrón común es sdemt<trimestre><año de dos dígitos>.dta
    file_format = 'dta'
    year_short = str(year)[-2:] # Obtiene los últimos dos dígitos del año, ej: 25 para 2025
    
    # Construir el nombre del directorio y del archivo
    dir_name = f"ENOE_{year}_{quarter}"
    file_name = f"ENOE_SDEMT{quarter}{year_short}.dta" # ej. sdemt225.dta
    
    file_path = os.path.join("Data", f"ENOE_{file_format}", dir_name, file_name)
    
    print(f"Buscando archivo en: {file_path}")

    # --- 2. Cargar los datos ---
    if not os.path.exists(file_path):
        print(f"❌ Error: No se encontró el archivo '{file_path}'.")
        print("   Verifica que la descarga se haya completado para este periodo.")
        return None
        
    try:
        # Usamos read_stata, que es muy rápido para archivos .dta
        df = pd.read_stata(file_path, convert_categoricals = False)
        print(f"✅ Archivo cargado exitosamente. {len(df)} registros leídos.")
    except Exception as e:
        print(f"❌ Error al leer el archivo Stata: {e}")
        return None

    # --- 3. Preparación y limpieza de datos ---
    # Asegurarse de que las columnas clave sean numéricas, manejando errores
    columnas_a_convertir = ['fac_tri', 'eda', 'sex']
    for col in columnas_a_convertir:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Eliminar filas donde la conversión falló o los datos son nulos
    df.dropna(subset=columnas_a_convertir, inplace=True)
    
    # --- 4. Realizar el cálculo ---
    print("Calculando población estimada por edad y género...")
    
    # Agrupar por edad (eda) y sexo (sex), y sumar el factor de expansión (fac_tri)
    poblacion = df.groupby(['eda', 'sex'])['fac_tri'].sum().reset_index()
    
    # --- 5. Mejorar la presentación ---
    # Mapear los códigos de 'sex' a etiquetas de texto para mayor claridad
    # Según el diccionario de datos de INEGI: 1 es Hombre, 2 es Mujer
    poblacion['sex'] = poblacion['sex'].map({1: 'Hombre', 2: 'Mujer'})
    
    # Renombrar columnas para un reporte más claro
    poblacion.rename(columns={
        'eda': 'Edad',
        'sex': 'Género',
        'fac_tri': 'Poblacion_Estimada'
    }, inplace=True)
    
    # Convertir edad a entero para una mejor visualización
    poblacion['Edad'] = poblacion['Edad'].astype(int)
    
    return poblacion

# --- Ejecución del Script ---
if __name__ == "__main__":
    # Define el año y trimestre que quieres analizar
    # Puedes cambiar estos valores para analizar otros periodos
    YEAR_A_ANALIZAR = 2025
    QUARTER_A_ANALIZAR = 2
    
    # Llamar a la función principal
    resultado_poblacion = analizar_poblacion_por_edad_y_genero(YEAR_A_ANALIZAR, QUARTER_A_ANALIZAR)
    
    # Imprimir los resultados si el análisis fue exitoso
    if resultado_poblacion is not None:
        print("\n--- Resultados del Análisis ---")
        print(f"Población estimada para {YEAR_A_ANALIZAR} T{QUARTER_A_ANALIZAR} (primeros 20 resultados):")
        # Usamos .to_string() para asegurar que se muestren todas las columnas bien alineadas
        print(resultado_poblacion.head(20).to_string())


--- Analizando datos para 2025 T2 ---
Buscando archivo en: Data\ENOE_dta\ENOE_2025_2\ENOE_SDEMT225.dta
✅ Archivo cargado exitosamente. 423744 registros leídos.
Calculando población estimada por edad y género...

--- Resultados del Análisis ---
Población estimada para 2025 T2 (primeros 20 resultados):
    Edad  Género  Poblacion_Estimada
0      0  Hombre            543800.0
1      0   Mujer            482893.0
2      1  Hombre            645797.0
3      1   Mujer            663950.0
4      2  Hombre            734343.0
5      2   Mujer            668793.0
6      3  Hombre            810301.0
7      3   Mujer            731215.0
8      4  Hombre            768016.0
9      4   Mujer            804740.0
10     5  Hombre            874860.0
11     5   Mujer            880538.0
12     6  Hombre            984654.0
13     6   Mujer            926439.0
14     7  Hombre           1008299.0
15     7   Mujer            939588.0
16     8  Hombre           1023076.0
17     8   Mujer            9927

In [7]:
import pandas as pd
import os
import numpy as np

def calcular_indicadores_ampliados(df):
    """
    Calcula una serie de indicadores demográficos y económicos a partir de los datos de la ENOE.
    
    Args:
        df (pd.DataFrame): El DataFrame ya cargado y limpio.
        
    Returns:
        dict: Un diccionario con todos los indicadores calculados.
    """
    print("\n--- Calculando Indicadores Ampliados ---")
    
    # --- Preparación y Filtros Base ---
    # Asegurar que las columnas adicionales para el cálculo sean numéricas
    columnas_numericas_adicionales = ['clase1', 'clase2', 'ingocup', 'ing_x_hrs', 'ent']
    for col in columnas_numericas_adicionales:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Filtro base: Entrevista completa ('0.0') y residente habitual o nuevo (1 o 3)
    df_base = df[(df['r_def'] == '0.0') & (df['c_res'].isin([1, 3]))].copy()
    
    # Filtro para población de 15 años y más
    df_15_y_mas = df_base[df_base['eda'].between(15, 98)].copy()
    
    # Filtro para población ocupada (para cálculos de ingreso)
    df_ocupada = df_15_y_mas[df_15_y_mas['clase2'] == 1].copy()

    # Diccionario para mapear códigos de entidad a nombres
    entidades = {
        1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche',
        5: 'Coahuila', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México',
        10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco',
        15: 'México', 16: 'Michoacán', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León',
        20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí',
        25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala',
        30: 'Veracruz', 31: 'Yucatán', 32: 'Zacatecas'
    }
    df_ocupada['ent_nombre'] = df_ocupada['ent'].map(entidades)


    # --- Inicio de Cálculos ---
    resultados = {}

    # 1. Total de la población (suma de todos los factores de expansión)
    resultados['total_poblacion'] = df_base['fac_tri'].sum()

    # 2. Total de hombres
    resultados['total_hombres'] = df_base[df_base['sex'] == 1]['fac_tri'].sum()

    # 3. Total de mujeres
    resultados['total_mujeres'] = df_base[df_base['sex'] == 2]['fac_tri'].sum()

    # 4. PEA Hombres
    df_pea = df_15_y_mas[df_15_y_mas['clase1'] == 1]
    resultados['pea_hombres'] = df_pea[df_pea['sex'] == 1]['fac_tri'].sum()

    # 5. PEA Mujeres
    resultados['pea_mujeres'] = df_pea[df_pea['sex'] == 2]['fac_tri'].sum()

    # Función para cálculo de promedio ponderado
    def weighted_average(df, value_col, weight_col):
        df_filtered = df.dropna(subset=[value_col, weight_col])
        if df_filtered.empty:
            return 0
        return np.average(df_filtered[value_col], weights=df_filtered[weight_col])

    # 6. Ingreso promedio mensual Mujer
    df_ocup_mujer = df_ocupada[df_ocupada['sex'] == 2]
    resultados['ingreso_prom_mensual_mujer'] = weighted_average(df_ocup_mujer, 'ingocup', 'fac_tri')

    # 7. Ingreso promedio mensual Hombre
    df_ocup_hombre = df_ocupada[df_ocupada['sex'] == 1]
    resultados['ingreso_prom_mensual_hombre'] = weighted_average(df_ocup_hombre, 'ingocup', 'fac_tri')

    # 8. Ingreso promedio mensual por estado
    resultados['ingreso_prom_mensual_estado'] = df_ocupada.groupby('ent_nombre').apply(
        lambda x: weighted_average(x, 'ingocup', 'fac_tri')
    )

    # 9. Ingreso por hora promedio
    resultados['ingreso_prom_hora_total'] = weighted_average(df_ocupada, 'ing_x_hrs', 'fac_tri')
    
    # 10. Ingreso promedio por hora (Hombre y Mujer)
    resultados['ingreso_prom_hora_mujer'] = weighted_average(df_ocup_mujer, 'ing_x_hrs', 'fac_tri')
    resultados['ingreso_prom_hora_hombre'] = weighted_average(df_ocup_hombre, 'ing_x_hrs', 'fac_tri')

    # 11. Masa salarial total
    resultados['masa_salarial_total'] = (df_ocupada['ingocup'] * df_ocupada['fac_tri']).sum()

    # 12. Masa salarial por Estado
    resultados['masa_salarial_estado'] = df_ocupada.groupby('ent_nombre').apply(
        lambda x: (x['ingocup'] * x['fac_tri']).sum()
    )

    return resultados

def analizar_poblacion_por_edad_y_genero(df):
    """
    Calcula la población estimada agrupada por edad y género.
    """
    print("\n--- Analizando Población por Edad y Género ---")
    
    # Agrupar por edad (eda) y sexo (sex), y sumar el factor de expansión (fac_tri)
    poblacion = df.groupby(['eda', 'sex'])['fac_tri'].sum().reset_index()
    
    # Mapear los códigos de 'sex' a etiquetas de texto
    poblacion['sex'] = poblacion['sex'].map({1: 'Hombre', 2: 'Mujer'})
    
    # Renombrar columnas
    poblacion.rename(columns={
        'eda': 'Edad', 'sex': 'Género', 'fac_tri': 'Poblacion_Estimada'
    }, inplace=True)
    
    poblacion['Edad'] = poblacion['Edad'].astype(int)
    return poblacion

# --- Ejecución del Script ---
if __name__ == "__main__":
    YEAR_A_ANALIZAR = 2025
    QUARTER_A_ANALIZAR = 2
    
    print(f"--- Iniciando análisis para {YEAR_A_ANALIZAR} T{QUARTER_A_ANALIZAR} ---")
    
    # --- 1. Carga de datos ---
    file_format = 'dta'
    year_short = str(YEAR_A_ANALIZAR)[-2:]
    dir_name = f"ENOE_{YEAR_A_ANALIZAR}_{QUARTER_A_ANALIZAR}"
    file_name = f"ENOE_SDEMT{QUARTER_A_ANALIZAR}{year_short}.dta"
    file_path = os.path.join("data", f"ENOE_{file_format}", dir_name, file_name)
    
    print(f"Buscando archivo en: {file_path}")
    if not os.path.exists(file_path):
        print(f"❌ Error: No se encontró el archivo. Ejecuta el script de descarga primero.")
    else:
        try:
            df_enoe = pd.read_stata(file_path, convert_categoricals=False)
            print(f"✅ Archivo cargado exitosamente. {len(df_enoe)} registros leídos.")
            
            # --- 2. Preparación general de datos ---
            columnas_base = ['fac_tri', 'eda', 'sex', 'r_def', 'c_res']
            for col in columnas_base:
                df_enoe[col] = pd.to_numeric(df_enoe[col], errors='coerce') if col not in ['r_def'] else df_enoe[col]
            df_enoe.dropna(subset=['fac_tri', 'eda', 'sex', 'c_res'], inplace=True)

            # --- 3. Ejecutar análisis y mostrar resultados ---
            
            # Análisis por edad y género
            poblacion_edad_genero = analizar_poblacion_por_edad_y_genero(df_enoe.copy())
            if poblacion_edad_genero is not None:
                print("\n--- Resultados: Población por Edad y Género (primeros 20) ---")
                print(poblacion_edad_genero.head(20).to_string())

            # Indicadores ampliados
            indicadores = calcular_indicadores_ampliados(df_enoe.copy())
            if indicadores:
                print("\n--- Resultados: Indicadores Demográficos y Económicos ---")
                print(f"1. Población Total Estimada: {indicadores['total_poblacion']:,.0f}")
                print(f"2. Total de Hombres: {indicadores['total_hombres']:,.0f}")
                print(f"3. Total de Mujeres: {indicadores['total_mujeres']:,.0f}")
                print(f"4. PEA Hombres: {indicadores['pea_hombres']:,.0f}")
                print(f"5. PEA Mujeres: {indicadores['pea_mujeres']:,.0f}")
                print(f"6. Ingreso Promedio Mensual (Mujer): ${indicadores['ingreso_prom_mensual_mujer']:,.2f}")
                print(f"7. Ingreso Promedio Mensual (Hombre): ${indicadores['ingreso_prom_mensual_hombre']:,.2f}")
                print("\n8. Ingreso Promedio Mensual por Estado:")
                print(indicadores['ingreso_prom_mensual_estado'].map('${:,.2f}'.format).to_string())
                print(f"\n9. Ingreso Promedio por Hora (Total): ${indicadores['ingreso_prom_hora_total']:,.2f}")
                print(f"10. Ingreso Promedio por Hora (Mujer): ${indicadores['ingreso_prom_hora_mujer']:,.2f}")
                print(f"    Ingreso Promedio por Hora (Hombre): ${indicadores['ingreso_prom_hora_hombre']:,.2f}")
                print(f"\n11. Masa Salarial Total Mensual: ${indicadores['masa_salarial_total']:,.2f}")
                print("\n12. Masa Salarial Mensual por Estado:")
                print(indicadores['masa_salarial_estado'].map('${:,.2f}'.format).to_string())

        except Exception as e:
            print(f"❌ Ocurrió un error durante el análisis: {e}")


--- Iniciando análisis para 2025 T2 ---
Buscando archivo en: data\ENOE_dta\ENOE_2025_2\ENOE_SDEMT225.dta
✅ Archivo cargado exitosamente. 423744 registros leídos.

--- Analizando Población por Edad y Género ---

--- Resultados: Población por Edad y Género (primeros 20) ---
    Edad  Género  Poblacion_Estimada
0      0  Hombre            543800.0
1      0   Mujer            482893.0
2      1  Hombre            645797.0
3      1   Mujer            663950.0
4      2  Hombre            734343.0
5      2   Mujer            668793.0
6      3  Hombre            810301.0
7      3   Mujer            731215.0
8      4  Hombre            768016.0
9      4   Mujer            804740.0
10     5  Hombre            874860.0
11     5   Mujer            880538.0
12     6  Hombre            984654.0
13     6   Mujer            926439.0
14     7  Hombre           1008299.0
15     7   Mujer            939588.0
16     8  Hombre           1023076.0
17     8   Mujer            992772.0
18     9  Hombre        

C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\1490822636.py:80: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultados['ingreso_prom_mensual_estado'] = df_ocupada.groupby('ent_nombre').apply(
C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\1490822636.py:95: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultados['masa_salarial_estado'] = df_ocupada.groupby('ent_nombre').apply(


In [8]:
import pandas as pd
import os
import numpy as np

def calcular_indicadores_enoe(df):
    """
    Calcula y muestra una serie de indicadores demográficos y económicos 
    a partir de un DataFrame de la ENOE.
    
    Args:
        df (pd.DataFrame): El DataFrame ya cargado.
    """
    print("\n--- Iniciando Cálculos de Indicadores ---")
    
    # --- 1. Limpieza y Preparación de Datos ---
    # Asegurar que las columnas clave para el filtrado y cálculo sean del tipo correcto.
    df['r_def'] = df['r_def'].astype(str).str.strip()
    df['c_res'] = pd.to_numeric(df['c_res'], errors='coerce')
    columnas_numericas = [
        'fac_tri', 'sex', 'eda', 'clase1', 'clase2', 
        'ingocup', 'ing_x_hrs', 'ent'
    ]
    for col in columnas_numericas:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- 2. Aplicación de Filtros Base ---
    # El universo para la mayoría de los cálculos son los residentes con entrevista completa.
    df_base = df[(df['r_def'] == '0.0') & (df['c_res'].isin([1, 3]))].copy()
    
    if df_base.empty:
        print("❌ Error Crítico: No se encontraron registros válidos (r_def='0.0' y c_res=1 o 3). No se pueden continuar los cálculos.")
        return

    # --- 3. Cálculos Principales y Presentación ---
    print("\n--- Resultados: Población General ---")
    
    # Población Total
    poblacion_total = df_base['fac_tri'].sum()
    print(f"1. Población Total Estimada: {poblacion_total:,.0f}")
    
    # Población por Género
    poblacion_hombres = df_base[df_base['sex'] == 1]['fac_tri'].sum()
    poblacion_mujeres = df_base[df_base['sex'] == 2]['fac_tri'].sum()
    print(f"2. Total de Hombres: {poblacion_hombres:,.0f}")
    print(f"3. Total de Mujeres: {poblacion_mujeres:,.0f}")

    # --- Cálculos Adicionales ---
    # Filtro para población de 15 años y más
    df_15_y_mas = df_base[df_base['eda'].between(15, 98)].copy()
    
    # PEA por Género
    df_pea = df_15_y_mas[df_15_y_mas['clase1'] == 1]
    pea_hombres = df_pea[df_pea['sex'] == 1]['fac_tri'].sum()
    pea_mujeres = df_pea[df_pea['sex'] == 2]['fac_tri'].sum()
    print(f"4. PEA Hombres: {pea_hombres:,.0f}")
    print(f"5. PEA Mujeres: {pea_mujeres:,.0f}")

    # --- Cálculos de Ingreso (sobre la población ocupada) ---
    df_ocupada = df_15_y_mas[df_15_y_mas['clase2'] == 1].copy()
    
    def weighted_average(df, value_col, weight_col):
        df_filtered = df.dropna(subset=[value_col, weight_col])
        if df_filtered.empty or df_filtered[weight_col].sum() == 0:
            return 0
        return np.average(df_filtered[value_col], weights=df_filtered[weight_col])

    # Ingreso Promedio Mensual por Género
    ing_prom_mujer = weighted_average(df_ocupada[df_ocupada['sex'] == 2], 'ingocup', 'fac_tri')
    ing_prom_hombre = weighted_average(df_ocupada[df_ocupada['sex'] == 1], 'ingocup', 'fac_tri')
    print(f"6. Ingreso Promedio Mensual (Mujer): ${ing_prom_mujer:,.2f}")
    print(f"7. Ingreso Promedio Mensual (Hombre): ${ing_prom_hombre:,.2f}")

    # Diccionario de Entidades
    entidades = {
        1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche',
        5: 'Coahuila', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México',
        10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco',
        15: 'México', 16: 'Michoacán', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León',
        20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí',
        25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala',
        30: 'Veracruz', 31: 'Yucatán', 32: 'Zacatecas'
    }
    df_ocupada['ent_nombre'] = df_ocupada['ent'].map(entidades)

    # Ingreso Promedio Mensual por Estado
    ing_prom_estado = df_ocupada.groupby('ent_nombre').apply(lambda x: weighted_average(x, 'ingocup', 'fac_tri'))
    print("\n8. Ingreso Promedio Mensual por Estado:")
    print(ing_prom_estado.map('${:,.2f}'.format).to_string())

    # Ingreso Promedio por Hora
    ing_hora_total = weighted_average(df_ocupada, 'ing_x_hrs', 'fac_tri')
    ing_hora_mujer = weighted_average(df_ocupada[df_ocupada['sex'] == 2], 'ing_x_hrs', 'fac_tri')
    ing_hora_hombre = weighted_average(df_ocupada[df_ocupada['sex'] == 1], 'ing_x_hrs', 'fac_tri')
    print(f"\n9. Ingreso Promedio por Hora (Total): ${ing_hora_total:,.2f}")
    print(f"10. Ingreso Promedio por Hora (Mujer): ${ing_hora_mujer:,.2f}")
    print(f"    Ingreso Promedio por Hora (Hombre): ${ing_hora_hombre:,.2f}")

    # Masa Salarial
    masa_salarial_total = (df_ocupada['ingocup'] * df_ocupada['fac_tri']).sum()
    masa_salarial_estado = df_ocupada.groupby('ent_nombre').apply(lambda x: (x['ingocup'] * x['fac_tri']).sum())
    print(f"\n11. Masa Salarial Total Mensual: ${masa_salarial_total:,.2f}")
    print("\n12. Masa Salarial Mensual por Estado:")
    print(masa_salarial_estado.map('${:,.2f}'.format).to_string())


# --- Ejecución del Script ---
if __name__ == "__main__":
    YEAR_A_ANALIZAR = 2025
    QUARTER_A_ANALIZAR = 2
    
    print(f"--- Iniciando análisis para {YEAR_A_ANALIZAR} T{QUARTER_A_ANALIZAR} ---")
    
    # --- 1. Carga de datos ---
    file_format = 'dta'
    year_short = str(YEAR_A_ANALIZAR)[-2:]
    dir_name = f"ENOE_{YEAR_A_ANALIZAR}_{QUARTER_A_ANALIZAR}"
    file_name = f"ENOE_SDEMT{QUARTER_A_ANALIZAR}{year_short}.dta"
    file_path = os.path.join("data", f"ENOE_{file_format}", dir_name, file_name)
    
    print(f"Buscando archivo en: {file_path}")
    if not os.path.exists(file_path):
        print(f"❌ Error: No se encontró el archivo. Ejecuta el script de descarga primero.")
    else:
        try:
            df_enoe = pd.read_stata(file_path, convert_categoricals=False)
            print(f"✅ Archivo cargado exitosamente. {len(df_enoe)} registros leídos.")
            
            # --- 2. Ejecutar análisis completo ---
            calcular_indicadores_enoe(df_enoe)

        except Exception as e:
            print(f"❌ Ocurrió un error durante el análisis: {e}")




--- Iniciando análisis para 2025 T2 ---
Buscando archivo en: data\ENOE_dta\ENOE_2025_2\ENOE_SDEMT225.dta
✅ Archivo cargado exitosamente. 423744 registros leídos.

--- Iniciando Cálculos de Indicadores ---

--- Resultados: Población General ---
1. Población Total Estimada: 130,575,786
2. Total de Hombres: 62,260,360
3. Total de Mujeres: 68,315,426
4. PEA Hombres: 36,046,346
5. PEA Mujeres: 25,018,659
6. Ingreso Promedio Mensual (Mujer): $5,066.40
7. Ingreso Promedio Mensual (Hombre): $6,341.43


C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\538922952.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ing_prom_estado = df_ocupada.groupby('ent_nombre').apply(lambda x: weighted_average(x, 'ingocup', 'fac_tri'))



8. Ingreso Promedio Mensual por Estado:
ent_nombre
Aguascalientes          $5,160.97
Baja California         $7,281.14
Baja California Sur    $11,825.56
Campeche                $8,138.65
Chiapas                 $5,340.33
Chihuahua               $8,440.37
Ciudad de México        $5,720.30
Coahuila                $9,338.23
Colima                  $7,383.05
Durango                 $7,485.46
Guanajuato              $4,647.14
Guerrero                $4,929.68
Hidalgo                 $5,761.37
Jalisco                 $5,215.58
Michoacán               $6,155.89
Morelos                 $2,937.55
México                  $3,288.29
Nayarit                 $8,751.47
Nuevo León              $8,585.35
Oaxaca                  $3,985.54
Puebla                  $4,665.10
Querétaro               $3,849.23
Quintana Roo            $8,347.20
San Luis Potosí         $4,713.56
Sinaloa                 $8,917.37
Sonora                  $6,350.36
Tabasco                 $6,428.02
Tamaulipas              $8,681

C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\538922952.py:100: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  masa_salarial_estado = df_ocupada.groupby('ent_nombre').apply(lambda x: (x['ingocup'] * x['fac_tri']).sum())


In [ ]:
# tablas a través del tiempo y por estados
# Datos ya trabajados
#

# 1. Porcentajes de ocupación por rango de edad
# 2. Ingresos por percentil 
#


,r_def,loc,mun,est,est_d_tri,est_d_men,ageb,t_loc_tri,t_loc_men,cd_a,...,scian,t_tra,emp_ppal,tue_ppal,trans_ppal,mh_fil2,mh_col,sec_ins,tipo,mes_cal
0,0.0,NaN,15.0,30.0,681.0,681.0,0.0,1.0,1.0,1.0,...,7.0,1.0,2.0,2.0,0.0,3.0,2.0,2.0,1.0,4.0
1,0.0,NaN,12.0,30.0,681.0,681.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
2,0.0,NaN,97.0,30.0,783.0,783.0,0.0,1.0,1.0,2.0,...,8.0,1.0,2.0,2.0,0.0,3.0,2.0,2.0,1.0,5.0
3,0.0,NaN,98.0,20.0,782.0,782.0,0.0,1.0,1.0,2.0,...,21.0,1.0,1.0,2.0,0.0,3.0,1.0,11.0,1.0,4.0
4,0.0,NaN,20.0,20.0,713.0,713.0,0.0,1.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423739,0.0,NaN,48.0,20.0,996.0,996.0,0.0,3.0,3.0,85.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
423740,0.0,NaN,1.0,20.0,517.0,517.0,0.0,4.0,4.0,86.0,...,8.0,1.0,2.0,2.0,0.0,3.0,2.0,2.0,1.0,4.0
423741,0.0,NaN,21.0,10.0,754.0,754.0,0.0,4.0,4.0,86.0,...,7.0,2.0,2.0,2.0,0.0,3.0,8.0,4.0,1.0,4.0
423742,0.0,NaN,32.0,20.0,845.0,845.0,0.0,4.0,4.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0


In [10]:
#df_enoe["r_def"].describe()
df_enoe["r_def"].value_counts()

r_def
0.0     422854
15.0       890
Name: count, dtype: int64

In [5]:
df



NameError: name 'df' is not defined

In [13]:
# versión con cambio en los nombres de las bases de datos 

import pandas as pd
import numpy as np
import os
from collections import defaultdict
import re
from datetime import datetime

# ----------------------------------------------------------------------
# FUNCIONES DE UTILIDAD
# ----------------------------------------------------------------------

def weighted_average(df, value_col, weight_col):
    """Calcula el promedio ponderado de una columna usando pesos (factores de expansión)."""
    df_filtered = df.dropna(subset=[value_col, weight_col])
    
    # Excluir valores de ingreso no válidos (generalmente negativos o no especificados, si se aplica)
    if value_col in ['ingocup', 'ing_x_hrs']:
        df_filtered = df_filtered[df_filtered[value_col] > 0].copy()
    
    if df_filtered.empty or df_filtered[weight_col].sum() == 0:
        return np.nan
    
    return np.average(df_filtered[value_col], weights=df_filtered[weight_col])

# Diccionario de Entidades para mapear códigos a nombres
ENTIDADES = {
    1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche',
    5: 'Coahuila', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México',
    10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco',
    15: 'México', 16: 'Michoacán', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León',
    20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí',
    25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala',
    30: 'Veracruz', 31: 'Yucatán', 32: 'Zacatecas'
}

def obtener_nombre_archivo(year, quarter, file_format='dta'):
    """Determina el nombre del archivo SDEMT según el periodo."""
    year_short = str(year)[-2:]
    
    # Periodo 1: 2005 T1 a 2018 T4 (Mayúsculas)
    if year <= 2018:
        base_name = f"SDEMT{quarter}{year_short}".upper()
    
    # Periodo 2: 2019 T1 a 2019 T4 (Minúsculas)
    elif year == 2019:
        base_name = f"sdemt{quarter}{year_short}".lower()
    
    # Periodo 3: 2020 T3 a 2022 T4 (Prefijo ENOEN_)
    elif 2020 <= year <= 2022:
        # 2020 T1 y T2 no tienen datos o son no oficiales (se manejan como "saltados" en el script principal)
        base_name = f"ENOEN_SDEMT{quarter}{year_short}".upper()
    
    # Periodo 4: 2023 T1 en adelante (Vuelve a Mayúsculas/Patrón consistente con el documento)
    else: # year >= 2023
        base_name = f"SDEMT{quarter}{year_short}".upper()
        
    dir_name = f"ENOE_{year}_{quarter}"
    file_name = f"{base_name}.{file_format}"
    # Asume que los archivos están en data/dta/ENOE_YYYY_Q/SDEMT...
    file_path = os.path.join("Data/ENOE_dta", dir_name, file_name) 
    
    return file_path

def pedir_rango_trimestral():
    """Pide al usuario el rango de años y trimestres para generar la serie de tiempo."""
    while True:
        try:
            print("\n--- Definición del Rango de la Serie de Tiempo ---")
            start_year = int(input("Ingrese el AÑO de inicio (e.g., 2018): "))
            start_quarter = int(input("Ingrese el TRIMESTRE de inicio (1 a 4): "))
            end_year = int(input("Ingrese el AÑO final (e.g., 2024): "))
            end_quarter = int(input("Ingrese el TRIMESTRE final (1 a 4): "))
            
            if not (1 <= start_quarter <= 4 and 1 <= end_quarter <= 4):
                raise ValueError("El trimestre debe ser un número entre 1 y 4.")
            
            start_date = datetime(start_year, start_quarter * 3 - 2, 1)
            end_date = datetime(end_year, end_quarter * 3 - 2, 1)

            if start_date > end_date:
                raise ValueError("El periodo de inicio debe ser anterior o igual al periodo final.")
                
            break
        except ValueError as e:
            print(f"Entrada inválida: {e}. Por favor, intente de nuevo.")
            
    # Generar la secuencia de trimestres
    periodos = []
    current_year = start_year
    current_quarter = start_quarter
    
    while current_year < end_year or (current_year == end_year and current_quarter <= end_quarter):
        
        # Manejo de trimestres faltantes (2020 T2 y T3 no oficiales/disponibles)
        if current_year == 2020 and current_quarter in [2, 3]:
            print(f"--- ⚠️ Saltando periodo {current_year} T{current_quarter} (No disponible o no oficial). ---")
            pass # No se añade el periodo a la lista para no intentar cargarlo.
            
        else:
            periodos.append((current_year, current_quarter))
            
        # Pasar al siguiente trimestre
        if current_quarter == 4:
            current_quarter = 1
            current_year += 1
        else:
            current_quarter += 1
            
    return periodos

# ----------------------------------------------------------------------
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO TRIMESTRAL
# ----------------------------------------------------------------------

def procesar_trimestre_enoe(year, quarter, file_format='dta'):
    """
    Carga, limpia y calcula indicadores clave a nivel nacional y estatal 
    para un trimestre específico.
    """
    periodo_str = f"{year} T{quarter}"
    print(f"\n--- ⏳ Procesando: {periodo_str} ---")

    # --- 1. Obtener Ruta y Ponderador ---
    file_path = obtener_nombre_archivo(year, quarter, file_format)
    
    # Determinar el campo ponderador correcto según el periodo 
    if year < 2020 or (year == 2020 and quarter < 3):
        PONDERATOR = 'FAC'
    else:
        PONDERATOR = 'FAC_TRI'
    
    # --- 2. Carga de Datos y Manejo de Errores (Debugging) ---
    if not os.path.exists(file_path):
        print(f"❌ Error Crítico: Archivo no encontrado en: {file_path}")
        return None, None
    
    try:
        if file_format == 'dta':
            # Se usa `encoding='latin-1'` si se encuentran problemas con codificación de texto
            df = pd.read_stata(file_path, convert_categoricals=False) 
        elif file_format == 'csv':
            df = pd.read_csv(file_path)
        else:
            raise ValueError("Formato de archivo no soportado.")
        
        if df.empty:
            print(f"❌ Error de Carga: Archivo encontrado, pero vacío: {file_path}")
            return None, None
            
        print(f"✅ Archivo cargado exitosamente. {len(df):,} registros. Ponderador: {PONDERATOR}")
        
    except Exception as e:
        print(f"❌ Ocurrió un error de lectura de datos en {file_path}: {e}")
        return None, None

    # --- 3. Limpieza y Preparación de Datos ---
    
    # Conversión de tipos de datos esenciales y estandarización de nombres
    columnas_requeridas = [
        PONDERATOR, 'sex', 'eda', 'clase1', 'clase2', 'c_res', 'r_def', 'ent',
        'ingocup', 'ing_x_hrs', 'pos_ocu', 'emp_ppal', 'sub_o' # Indicadores estratégicos
    ]
    
    for col in columnas_requeridas:
        if col not in df.columns:
            # Añadir columna con NaN/0 si falta, para evitar errores en cálculos posteriores (excepto ponderador)
            if col == PONDERATOR:
                 print(f"❌ Error Crítico: Columna de ponderador '{PONDERATOR}' no encontrada.")
                 return None, None
            df[col] = np.nan if col not in ['r_def', 'c_res'] else 0
            print(f"⚠️ Columna '{col}' no encontrada. Se añadió con NaN/0 para proseguir.")

    # Conversión de tipos
    df['r_def'] = df['r_def'].astype(str).str.strip()
    for col in ['sex', 'eda', 'clase1', 'clase2', 'c_res', 'ent', 'pos_ocu', 'emp_ppal', 'sub_o']:
         # Convertir a numérico, forzando errores a NaN, luego a entero (si es posible)
         df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    for col in ['ingocup', 'ing_x_hrs', PONDERATOR]:
         df[col] = pd.to_numeric(df[col], errors='coerce')


    # CRITERIO GENERAL DE FILTRADO (POBLACIÓN DE 15 AÑOS Y MÁS)
    # R_DEF='00' y (C_RES=1 o 3) y (EDA>=15 y EDA<=98) [cite: 148]
    
    # 1. Población total residente
    df_base = df[(df['r_def'] == '00') & (df['c_res'].isin([1, 3]))].copy()

    # 2. Población en Edad de Trabajar (PET) 15 años y más
    df_15_y_mas = df_base[df_base['eda'].between(15, 98)].copy()
    
    if df_15_y_mas.empty:
        print("❌ Error de Filtro: No se encontraron registros válidos después del filtro PET.")
        return None, None
    
    # Asignación de nombres de estado
    df_base['ent_nombre'] = df_base['ent'].map(ENTIDADES)
    df_15_y_mas['ent_nombre'] = df_15_y_mas['ent'].map(ENTIDADES)
    
    # ------------------------------------------------------------------
    # --- 4. CÁLCULOS A NIVEL NACIONAL ---
    # ------------------------------------------------------------------
    
    # Subconjuntos basados en campos precodificados y el criterio general [cite: 147]
    df_pea = df_15_y_mas[df_15_y_mas['clase1'] == 1].copy()      
    df_pnea = df_15_y_mas[df_15_y_mas['clase1'] == 2].copy()
    df_ocupada = df_15_y_mas[df_15_y_mas['clase2'] == 1].copy() 

    datos_nacional = {
        # Identificadores de Tiempo
        'year': year,
        'quarter': quarter,
        
        # 1. Población
        'pob_total': df_base[PONDERATOR].sum(),
        'pob_15_y_mas': df_15_y_mas[PONDERATOR].sum(), # [cite: 162]
        'pob_hombres_total': df_base[df_base['sex'] == 1][PONDERATOR].sum(),
        'pob_mujeres_total': df_base[df_base['sex'] == 2][PONDERATOR].sum(),
        
        # 2. PEA y PNEA
        'pea_total': df_pea[PONDERATOR].sum(),
        'pea_hombres': df_pea[df_pea['sex'] == 1][PONDERATOR].sum(),
        'pea_mujeres': df_pea[df_pea['sex'] == 2][PONDERATOR].sum(),
        'pnea_total': df_pnea[PONDERATOR].sum(), # [cite: 163]
        
        # Indicadores Estratégicos (CLASE2 y CLASE1) [cite: 162, 163]
        'ocupada_total': df_ocupada[PONDERATOR].sum(),
        'desocupada_total': df_15_y_mas[df_15_y_mas['clase2'] == 2][PONDERATOR].sum(),
        'pnea_disponible': df_15_y_mas[df_15_y_mas['clase2'] == 3][PONDERATOR].sum(),
        'pnea_no_disponible': df_15_y_mas[df_15_y_mas['clase2'] == 4][PONDERATOR].sum(),
        
        # Indicadores Estratégicos (POSICIÓN EN LA OCUPACIÓN - Ocupados) [cite: 163]
        'subordinados_remunerados': df_ocupada[df_ocupada['pos_ocu'] == 1][PONDERATOR].sum(),
        'empleadores': df_ocupada[df_ocupada['pos_ocu'] == 2][PONDERATOR].sum(),
        'cuenta_propia': df_ocupada[df_ocupada['pos_ocu'] == 3][PONDERATOR].sum(),
        'trabajadores_no_remunerados': df_ocupada[df_ocupada['pos_ocu'] == 4][PONDERATOR].sum(),
        
        # Indicadores Estratégicos (CONDICIÓN DE INFORMALIDAD - Ocupados) [cite: 169]
        'ocupacion_formal': df_ocupada[df_ocupada['emp_ppal'] == 2][PONDERATOR].sum(),
        'ocupacion_informal': df_ocupada[df_ocupada['emp_ppal'] == 1][PONDERATOR].sum(),
        
        # Indicador Estratégico (SUBOCUPACIÓN - Ocupados)
        'subocupacion': df_ocupada[df_ocupada['sub_o'] == 1][PONDERATOR].sum(),
        
        # 3. Ingreso Promedio
        'ing_prom_mes_total': weighted_average(df_ocupada, 'ingocup', PONDERATOR),
        'ing_prom_hora_total': weighted_average(df_ocupada, 'ing_x_hrs', PONDERATOR),
    }
    
    # ------------------------------------------------------------------
    # --- 5. CÁLCULOS A NIVEL ESTATAL ---
    # ------------------------------------------------------------------
    
    datos_estatal = defaultdict(list)
    
    for ent_code, ent_name in ENTIDADES.items():
        # Filtros base por Estado (Criterio General)
        df_base_est = df_base[df_base['ent'] == ent_code].copy()
        df_15_y_mas_est = df_15_y_mas[df_15_y_mas['ent'] == ent_code].copy()
        
        # Subconjuntos Estatales (basados en precodificados y el filtro base estatal)
        df_pea_est = df_15_y_mas_est[df_15_y_mas_est['clase1'] == 1].copy()
        df_pnea_est = df_15_y_mas_est[df_15_y_mas_est['clase1'] == 2].copy()
        df_ocupada_est = df_15_y_mas_est[df_15_y_mas_est['clase2'] == 1].copy()
        
        # Recolección de datos
        datos_estatal['year'].append(year)
        datos_estatal['quarter'].append(quarter)
        datos_estatal['ent_code'].append(ent_code)
        datos_estatal['ent_nombre'].append(ent_name)
        
        # Población
        datos_estatal['pob_total'].append(df_base_est[PONDERATOR].sum())
        datos_estatal['pob_15_y_mas'].append(df_15_y_mas_est[PONDERATOR].sum())
        datos_estatal['pob_hombres_total'].append(df_base_est[df_base_est['sex'] == 1][PONDERATOR].sum())
        datos_estatal['pob_mujeres_total'].append(df_base_est[df_base_est['sex'] == 2][PONDERATOR].sum())
        
        # PEA y PNEA
        datos_estatal['pea_total'].append(df_pea_est[PONDERATOR].sum())
        datos_estatal['pnea_total'].append(df_pnea_est[PONDERATOR].sum())
        
        # Indicadores Estratégicos (CLASE2 y CLASE1)
        datos_estatal['ocupada_total'].append(df_ocupada_est[PONDERATOR].sum())
        datos_estatal['desocupada_total'].append(df_15_y_mas_est[df_15_y_mas_est['clase2'] == 2][PONDERATOR].sum())
        datos_estatal['pnea_disponible'].append(df_15_y_mas_est[df_15_y_mas_est['clase2'] == 3][PONDERATOR].sum())
        datos_estatal['pnea_no_disponible'].append(df_15_y_mas_est[df_15_y_mas_est['clase2'] == 4][PONDERATOR].sum())
        
        # Indicadores Estratégicos (POSICIÓN EN LA OCUPACIÓN - Ocupados)
        datos_estatal['subordinados_remunerados'].append(df_ocupada_est[df_ocupada_est['pos_ocu'] == 1][PONDERATOR].sum())
        datos_estatal['empleadores'].append(df_ocupada_est[df_ocupada_est['pos_ocu'] == 2][PONDERATOR].sum())
        datos_estatal['cuenta_propia'].append(df_ocupada_est[df_ocupada_est['pos_ocu'] == 3][PONDERATOR].sum())
        datos_estatal['trabajadores_no_remunerados'].append(df_ocupada_est[df_ocupada_est['pos_ocu'] == 4][PONDERATOR].sum())

        # Indicadores Estratégicos (CONDICIÓN DE INFORMALIDAD - Ocupados)
        datos_estatal['ocupacion_formal'].append(df_ocupada_est[df_ocupada_est['emp_ppal'] == 2][PONDERATOR].sum())
        datos_estatal['ocupacion_informal'].append(df_ocupada_est[df_ocupada_est['emp_ppal'] == 1][PONDERATOR].sum())
        
        # Indicador Estratégico (SUBOCUPACIÓN - Ocupados)
        datos_estatal['subocupacion'].append(df_ocupada_est[df_ocupada_est['sub_o'] == 1][PONDERATOR].sum())

        # Ingreso Promedio
        datos_estatal['ing_prom_mes_total'].append(weighted_average(df_ocupada_est, 'ingocup', PONDERATOR))
        datos_estatal['ing_prom_hora_total'].append(weighted_average(df_ocupada_est, 'ing_x_hrs', PONDERATOR))

    df_estatal_trimestre = pd.DataFrame(datos_estatal)
    
    return pd.Series(datos_nacional), df_estatal_trimestre

# ----------------------------------------------------------------------
# EJECUCIÓN DEL SCRIPT Y CONSOLIDACIÓN DE SERIES DE TIEMPO
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    periodos = pedir_rango_trimestral()
    
    # Inicialización para la consolidación
    resultados_nacionales = []
    resultados_estatales = []

    print(f"\n===========================================================")
    print(f"  INICIANDO PROCESAMIENTO DE {len(periodos)} TRIMESTRES")
    print(f"===========================================================")

    # Bucle principal para procesar cada trimestre
    for year, quarter in periodos:
        
        df_nacional, df_estatal = procesar_trimestre_enoe(year, quarter)
        
        if df_nacional is not None and df_estatal is not None:
            resultados_nacionales.append(df_nacional)
            resultados_estatales.append(df_estatal)
        else:
            # Manejo explícito de trimestres sin datos (se añade una fila con NA)
            # Esto se asegura de mantener la continuidad de la serie de tiempo.
            periodo_na = {'year': year, 'quarter': quarter}
            
            # Serie Nacional con NA
            serie_na_nacional = pd.Series(periodo_na)
            # Se añaden las columnas faltantes (variables calculadas) con NaN
            if resultados_nacionales:
                # Usar la estructura de la primera serie de tiempo para rellenar los NaNs
                for col in resultados_nacionales[0].index:
                    if col not in serie_na_nacional:
                         serie_na_nacional[col] = np.nan
            resultados_nacionales.append(serie_na_nacional)
            
            # DataFrame Estatal con NA
            df_na_estatal = pd.DataFrame(periodo_na, index=range(1, 33)) # 32 estados
            df_na_estatal['ent_code'] = df_na_estatal.index
            df_na_estatal['ent_nombre'] = df_na_estatal['ent_code'].map(ENTIDADES)
            # Rellenar todas las columnas de variables con NaN
            if resultados_estatales:
                 # Usar la estructura del primer DataFrame estatal para rellenar los NaNs
                for col in resultados_estatales[0].columns:
                    if col not in df_na_estatal.columns:
                        df_na_estatal[col] = np.nan
            resultados_estatales.append(df_na_estatal)
            
            print(f"--- 🚫 Se agregó NA/NaN para {year} T{quarter} y se prosigue. ---")
            

    # --- 6. CONSOLIDACIÓN DE BASES DE DATOS ---

    # 1. Serie de Tiempo Nacional
    df_serie_nacional = pd.DataFrame(resultados_nacionales).reset_index(drop=True)
    df_serie_nacional['periodo'] = df_serie_nacional['year'].astype(str) + '-T' + df_serie_nacional['quarter'].astype(str)
    df_serie_nacional.set_index('periodo', inplace=True)

    print("\n===========================================================")
    print("      ✅ BASE DE SERIE DE TIEMPO NACIONAL CREADA")
    print("      (Incluye nuevos indicadores estratégicos)")
    print("===========================================================")
    print(df_serie_nacional.head())
    # Opcional: df_serie_nacional.to_csv("serie_tiempo_nacional_estrat.csv")


    # 2. Serie de Tiempo Estatal
    df_serie_estatal = pd.concat(resultados_estatales, ignore_index=True)
    df_serie_estatal['periodo'] = df_serie_estatal['year'].astype(str) + '-T' + df_serie_estatal['quarter'].astype(str)
    
    print("\n===========================================================")
    print("      ✅ BASE DE SERIE DE TIEMPO ESTATAL CREADA")
    print("      (Incluye nuevos indicadores estratégicos)")
    print("===========================================================")
    print(df_serie_estatal.head())
    # Opcional: df_serie_estatal.to_csv("serie_tiempo_estatal_estrat.csv")


--- Definición del Rango de la Serie de Tiempo ---
--- ⚠️ Saltando periodo 2020 T2 (No disponible o no oficial). ---
--- ⚠️ Saltando periodo 2020 T3 (No disponible o no oficial). ---

  INICIANDO PROCESAMIENTO DE 80 TRIMESTRES

--- ⏳ Procesando: 2005 T1 ---
✅ Archivo cargado exitosamente. 424,007 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2005 T1 y se prosigue. ---

--- ⏳ Procesando: 2005 T2 ---
✅ Archivo cargado exitosamente. 428,727 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2005 T2 y se prosigue. ---

--- ⏳ Procesando: 2005 T3 ---
✅ Archivo cargado exitosamente. 421,751 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2005 T3 y se prosigue. ---

--- ⏳ Procesando: 2005 T4 ---
✅ Archivo cargado exitosamente. 423,757 registros. Ponderador: FAC
❌ Error Crítico: Columna de pondera

C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\720325823.py:142: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


✅ Archivo cargado exitosamente. 407,725 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2009 T1 y se prosigue. ---

--- ⏳ Procesando: 2009 T2 ---


C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\720325823.py:142: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


✅ Archivo cargado exitosamente. 405,529 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2009 T2 y se prosigue. ---

--- ⏳ Procesando: 2009 T3 ---
✅ Archivo cargado exitosamente. 402,919 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2009 T3 y se prosigue. ---

--- ⏳ Procesando: 2009 T4 ---
✅ Archivo cargado exitosamente. 403,862 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2009 T4 y se prosigue. ---

--- ⏳ Procesando: 2010 T1 ---
✅ Archivo cargado exitosamente. 406,797 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2010 T1 y se prosigue. ---

--- ⏳ Procesando: 2010 T2 ---
✅ Archivo cargado exitosamente. 408,164 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/Na

C:\Users\rodri\AppData\Local\Temp\ipykernel_12592\720325823.py:142: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


✅ Archivo cargado exitosamente. 392,937 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2013 T1 y se prosigue. ---

--- ⏳ Procesando: 2013 T2 ---
✅ Archivo cargado exitosamente. 393,107 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2013 T2 y se prosigue. ---

--- ⏳ Procesando: 2013 T3 ---
✅ Archivo cargado exitosamente. 394,472 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2013 T3 y se prosigue. ---

--- ⏳ Procesando: 2013 T4 ---
✅ Archivo cargado exitosamente. 400,354 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/NaN para 2013 T4 y se prosigue. ---

--- ⏳ Procesando: 2014 T1 ---
✅ Archivo cargado exitosamente. 404,014 registros. Ponderador: FAC
❌ Error Crítico: Columna de ponderador 'FAC' no encontrada.
--- 🚫 Se agregó NA/Na